In [1]:
from sklearn.neural_network import MLPClassifier
#from sklearn.datasets import fetch_openml
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# take the dataset
#X, y = fetch_openml("mnist_784", return_X_y = True)
D = load_digits()
X, y = D.data, D.target

In [24]:
# create train & test datasets
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=.7)

# create MPL
model = MLPClassifier(hidden_layer_sizes=(10,))

# TRAIN 
model.fit(X_train, y_train)

# test the model
y_pred = model.predict(X_test)

/home/ygifoom/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [25]:
# benchmark
acc = accuracy_score(y_true = y_test, y_pred = y_pred)

print(acc)

0.9444444444444444


# With torch

In [1]:
import torch
from torch.utils.data import dataloader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch import nn
import torchmetrics

/home/ygifoom/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [15]:
# importing the dataset
training_data = datasets.FashionMNIST(
    root="pytorch_datasets",
    train=True,
    download=True,
    transform = ToTensor() # transforming the data insto tensor
    
    
)
test_data = datasets.FashionMNIST(
    root="pytorch_datasets",
    train=False,
    download=True,
    transform = ToTensor() # transforming the data insto tensor
    
)

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [16]:
class OurMlp(nn.Module):
    def __init__(self):
        super().__init__()
        # specify input size and output size
        self.mlp = nn.Sequential(
        nn.Linear(28*28, 50),
        nn.Sigmoid(),
        nn.Linear(50, 100),
        nn.Sigmoid(),
        nn.Linear(100, 50),
        nn.Sigmoid(),
        nn.Linear(50, 10)
        )
        self.flatten = nn.Flatten()
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.mlp(x)
        return logits

# instance of the model
model = OurMlp()
model.to(device)

# define the hyperparameters
epochs = 2
batch_size = 16
learning_rate = 0.001

# define the loss function
loss_fn = nn.CrossEntropyLoss()

# define the optimazer (adamW is a good optimizer)
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

# create dataloader
train_dataloader = dataloader(training_data, batch_size= batch_size)
test_dataloader = dataloader(test_data, batch_size=batch_size)

# define accuracy matric, task can be multiclass, multilabel.... (labels are inside classes)
metric = torchmetrics.Accuracy(task="multiclass", num_classes=10)

# define training loop
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader)
    
    for batch, (X,y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        
        # compute the prediction and the loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # adjusting the weights 
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        # print some info
        if batch % 20 == 0:
            loss_v = current_batch =loss.item(), (batch+1) * len(X)        
            print(f"loss: {loss_v} [{current_batch}/{size}]")
            acc = metric(pred, y)
            print(f"accuracy of current batch: {acc}")
    
    acc = metric.compute()
    print(f"Final accuracy: {acc}")
    metric.reset()

# and now for the actual testing loop
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader)
    
    # disble updating of weights for this is test
    with torch.no_grad():
        for X,y in dataloader:
            # move data to correct device
            X = X.to(device)
            y = y.to(device)
            
            # get predictions
            pred = model(X)
            
            # final accuracy
            acc = metric(pred, y)
    
    acc = metric.compute()
    print(f"final accuracy score: {acc}")
    metric.reset()
    

# train
for epoch in range(epochs):
    print(f"Epoch: {epoch}")
    train_loop(train_dataloader, model, loss_fn, optimizer)
test_loop(test_dataloader, model, loss_fn, optimizer)

acc = metric.compute()
print(f"final accuracy: {acc}")
print(f"done!")
            

TypeError: 'module' object is not callable

In [ ]:
# different way to specify the layers

class OurMlp(nn.Module):
    def __init__(self):
        
        # specify input size and output size
        self.input_layer = nn.Linear(28*28, 50)
        self.activation = nn.Sigmoid()
        self.hidden1 = nn.Linear(50, 100)
        self.hidden2 = nn.Linear(100, 50)
        self.output_layer = nn.Linear(50, 10)
        self.flatten = nn.Flatten()
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.flatten(x)
        logits = self.mlp(x)
        return logits

# instance of the model
model = OurMlp().to(device)

# define the hyperparameters
epochs = 2
batch_size = 16
learning_rate = 0.001

# define the loss function
loss_fn = nn.CrossEntropyLoss()

# define the optimazer (adamW is a good optimizer)
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

# create dataloader
train_dataloader = dataloader(training_data, batch_size= batch_size)
test_dataloader = dataloader(test_data, batch_size=batch_size)

# define accuracy matric, task can be multiclass, multilabel.... (labels are inside classes)
metric = torchmetrics.Accuracy(task="multiclass", num_classes=10)

# define training loop
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader)
    
    for batch, (X,y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        
        # compute the prediction and the loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # adjusting the weights 
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        # print some info
        if batch % 20 == 0:
            loss_v = current_batch =loss.item(), (batch+1) * len(X)        
            print(f"loss: {loss_v} [{current_batch}/{size}]")
            acc = metric(pred, y)
            print(f"accuracy of current batch: {acc}")
    
    acc = metric.compute()
    print(f"Final accuracy: {acc}")
    metric.reset()

# and now for the actual testing loop
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader)
    
    # disble updating of weights for this is test
    with torch.no_grad():
        for X,y in dataloader:
            # move data to correct device
            X = X.to(device)
            y = y.to(device)
            
            # get predictions
            pred = model(X)
            
            # final accuracy
            acc = metric(pred, y)
    
    acc = metric.compute()
    print(f"final accuracy score: {acc}")
    metric.reset()
    

# train
for epoch in range(epochs):
    print(f"Epoch: {epoch}")
    train_loop(train_dataloader, model, loss_fn, optimizer)
test_loop(test_dataloader, model, loss_fn, optimizer)

acc = metric.compute()
print(f"final accuracy: {acc}")
print(f"done!")
            